# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [12]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,puerto natales,-51.7236,-72.4875,271.13,82,96,1.08,CL,1692322288
1,1,hermanus,-34.4187,19.2345,284.71,85,3,5.76,ZA,1692322522
2,2,ushuaia,-54.8000,-68.3000,274.96,64,0,2.06,AR,1692322301
3,3,albany,42.6001,-73.9662,295.51,83,4,0.89,US,1692322486
4,4,grytviken,-54.2811,-36.5092,265.43,77,15,8.04,GS,1692322493


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.scatter(
    x='Lng',
    y='Lat',
    c='Humidity',
    size='Humidity',
    title='Humidity by Cities',
)

# Display the map
map_plot

:Scatter   [Lng]   (Lat,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
preferred_city_df = city_data_df[city_data_df['Humidity'] > 80]

# Drop any rows with null values
preferred_city_df.dropna(inplace=True)

# Display sample data
preferred_city_df.head()

/var/folders/sp/gm7ynjg50fnfmqtv0c8mygmc0000gn/T/ipykernel_8327/3711166674.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  preferred_city_df.dropna(inplace=True)


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,puerto natales,-51.7236,-72.4875,271.13,82,96,1.08,CL,1692322288
1,1,hermanus,-34.4187,19.2345,284.71,85,3,5.76,ZA,1692322522
3,3,albany,42.6001,-73.9662,295.51,83,4,0.89,US,1692322486
9,9,tarfaya,27.9358,-12.9218,294.37,88,38,8.62,MA,1692322525
13,13,saint-girons,42.9167,1.1667,292.73,88,63,2.01,FR,1692322526


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
selected_columns = ['City', 'Country','Lat','Lng','Humidity']

hotel_df=preferred_city_df[selected_columns].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name']=''

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,puerto natales,CL,-51.7236,-72.4875,82,
1,hermanus,ZA,-34.4187,19.2345,85,
3,albany,US,42.6001,-73.9662,83,
9,tarfaya,MA,27.9358,-12.9218,88,
13,saint-girons,FR,42.9167,1.1667,88,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [23]:
# Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel"


params = {
        "categories": categories,
        "radius": radius,
        "apiKey": geoapify_key,
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    latitude = row['Lat']
    longitude = row['Lng']

    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
puerto natales - nearest hotel: Dorotea Patagonia Hostel
hermanus - nearest hotel: Aloe guest house
albany - nearest hotel: No hotel found
tarfaya - nearest hotel: Hotel Tarfaya
saint-girons - nearest hotel: La Souleille des Lannes
vorgashor - nearest hotel: София
monrovia - nearest hotel: Bella Casa Hotel
arraial do cabo - nearest hotel: No hotel found
bethel - nearest hotel: Hampton Inn Danbury
gergebil' - nearest hotel: Гарден Отель
tiksi - nearest hotel: Арктика
el chorrillo - nearest hotel: Hostal Eden
villa rica - nearest hotel: Comfort Inn & Suites
hawaiian paradise park - nearest hotel: No hotel found
acarau - nearest hotel: No hotel found
ribeira grande - nearest hotel: Hospedaria JSF
sarigerme - nearest hotel: Hotel Sevgi
yaya - nearest hotel: No hotel found
west island - nearest hotel: Cocos Village Bungalows
lejanias - nearest hotel: Hel llano real
hamilton - nearest hotel: North Vista Manor
pacific grove - nearest hotel: Pacific Grove Inn
st. john's -

,City,Country,Lat,Lng,Humidity,Hotel Name
0,puerto natales,CL,-51.7236,-72.4875,82,Dorotea Patagonia Hostel
1,hermanus,ZA,-34.4187,19.2345,85,Aloe guest house
3,albany,US,42.6001,-73.9662,83,No hotel found
9,tarfaya,MA,27.9358,-12.9218,88,Hotel Tarfaya
13,saint-girons,FR,42.9167,1.1667,88,La Souleille des Lannes
...,...,...,...,...,...,...
570,bafoulabe,ML,13.8065,-10.8321,94,No hotel found
571,kimbe,PG,-5.5502,150.1429,89,Kimbe Bay Hotel
572,mersa matruh,EG,31.3525,27.2453,83,صقور
574,puerto madryn,AR,-42.7692,-65.0385,89,Hotel Gran Palace


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [30]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    hover_cols=['Hotel Name', 'Country']
)

# Display the map plot
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Hotel Name,Country)